In [3]:
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
import os
HF_API_TOKEN = os.getenv('HF_API_TOKEN')

# Data

In [4]:
from datasets import load_dataset

dataset = load_dataset("yqzheng/semeval2014_restaurants")

In [5]:
import pandas as pd
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
train.shape, test.shape

((3608, 5), (1120, 5))

In [6]:
def merge_jsons(json_list):
    result = {}
    for j in json_list:
        result.update(j)
    return result

def create_json(df):
    df['json'] = df.apply(lambda row: {row['aspect']: row['label']} , axis=1)
    return df.groupby('text')['json'].agg(merge_jsons).reset_index()

In [7]:
train_json = create_json(train)
test_json = create_json(test)
train_json.head(10)

,text,json
0,"$160 for 2 filets, 2 sides, an appetizer and d...","{'filets': 0, 'sides': 0, 'appetizer': 0, 'dri..."
1,$20 for all you can eat sushi cannot be beaten.,{'sushi': 0}
2,$20 gets you unlimited sushi of a very high qu...,"{'sushi': 1, 'sushi places': 1, 'quality': 1}"
3,"$6 and there is much tasty food, all of it fre...",{'food': 1}
4,"($200 for 2 glasses of champagne, not too expe...","{'glasses of champagne': -1, 'bottle of wine':..."
5,(Always ask the bartender for the SEASONAL bee...,"{'SEASONAL beer': 1, 'bartender': 0}"
6,(and I have eaten my share) Which impresses me...,{'serve': 1}
7,"(food was delivered by a busboy, not waiter) W...","{'food': 0, 'busboy': -1, 'waiter': -1, 'chees..."
8,- the bread at the beginning is super tasty an...,"{'bread': 1, 'pizza': 1, 'margarite pizza with..."
9,20 minutes for our reservation but it gave us ...,"{'reservation': -1, 'cocktails': 1, 'surroundi..."


# Evaluation

In [11]:
def calc_f1(tp: int, fp: int, fn: int) -> float:
    if tp + fp == 0: 
        precision = 0
    else:
        precision = tp / (tp + fp)
    
    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)
    
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2.0 * (precision * recall) / (precision + recall)
    return f1_score

def validate_f1(example: dict, pred: dict) -> float:
    tp = sum(1 for k, v in example.items() if pred.get(k) == v)
    fp = sum(1 for k, v in pred.items() if example.get(k) != v)
    fn = sum(1 for k, v in example.items() if k not in pred.keys())
    return calc_f1(tp, fp, fn)

# Original Prompt

In [9]:
import requests
API_URL = "https://api-inference.huggingface.co/models/kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants"
headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}
def query(prompt):
    payload = {
        "inputs": prompt,
        "options": {"wait_for_model": True}
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [12]:
orig_prompt_template = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
Positive example 1-
input: With the great variety on the menu , I eat here often and never get bored.
output: menu:positive
Positive example 2- 
input: Great food, good size menu, great service and an unpretensious setting.
output: food:positive, menu:positive, service:positive, setting:positive
Negative example 1-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it.
output: toast:negative, mayonnaise:negative, bacon:negative, ingredients:negative, plate:negative
Negative example 2-
input: The seats are uncomfortable if you are sitting against the wall on wooden benches.
output: seats:negative
Neutral example 1-
input: I asked for seltzer with lime, no ice.
output: seltzer with lime:neutral
Neutral example 2-
input: They wouldnt even let me finish my glass of wine before offering another.
output: glass of wine:neutral
Now complete the following example-
input: {}
output:"""

In [13]:
res = query(orig_prompt_template.format(test.loc[0, 'text']))
res

[{'generated_text': 'bread:positive'}]

In [14]:
def convert_sentiment(text):
    if text == 'positive':
        return 1
    elif text == 'negative':
        return -1
    elif text == 'neutral':
        return 0
    else:
        return None

def convert_dict(res):
    return {l.split(":")[0].strip(): convert_sentiment(l.split(":")[1].strip()) for l in res[0]['generated_text'].split(",") if ':' in l}

In [15]:
f1s = []
for _, row in test_json.iloc[:100].iterrows():
    res = query(orig_prompt_template.format(row['text']))
    pred = convert_dict(res)
    f1 = validate_f1(row['json'], pred)
    f1s.append(f1)
sum(f1s) / len(f1s)

0.7375714285714285

# Normal DSPy

In [16]:
train_dspy = [dspy.Example(review=row['text'], aspect_with_label=row['json']).with_inputs('review') for _, row in train_json.iterrows()]
test_dspy = [dspy.Example(review=row['text'], aspect_with_label=row['json']).with_inputs('review') for _, row in test_json.iterrows()]

In [17]:
import dspy
import os
import requests

class HFServerless(dspy.dsp.modules.LM):
    def __init__(
        self,
        model: str,
        api_key: str,
        **kwargs):
        """
        Wrapper around HuggingFace's Inference API (Serverless)
        """
        super().__init__(model)
        self.api_url = "https://api-inference.huggingface.co/models/" + model
        self.api_key = api_key
    
    def basic_request(self, prompt, **kwargs):
        headers = {"Authorization": f"Bearer {self.api_key}"}
        payload = {
            "inputs": prompt,
            "options": {"wait_for_model": True}
        }
        response = requests.post(self.api_url, headers=headers, json=payload)
        response.raise_for_status()
        response_text = response.json()[0]['generated_text']

        self.history.append(
            {
                "prompt": prompt,
                "response": {
                    "choices": [{'text': response_text}]
                }
            }
        )
        
        return response.json()

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        response = self.request(prompt, **kwargs)
        return [response[0]['generated_text']]

hf = HFServerless(model="kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants", api_key=os.getenv('HF_API_TOKEN'))
dspy.settings.configure(lm=hf)

In [49]:
class Review2Aspects(dspy.Signature):
    """
    Identify aspects and their sentiments from a customer review. The aspects must be words or phrases in the review.
    The response should be a Python dictionary, where each key is an aspect and the value is a sentiment label.
    A label of 1 indicates positive sentiment, 0 indicates neutral sentiment, and -1 indicates negative sentiment.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspects_with_label: dict = dspy.OutputField(format=dict, desc=dedent("""
        a single Python dictionary, where each key is an aspect and the value is the label,
        with label 1 indicating positive sentiment, 0 indicating neutral sentiment, and -1
        indicating negative sentiment.
        """))

In [50]:
class DSPyInstructABSA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_aspects = dspy.ChainOfThought(Review2Aspects)

    def forward(self, review):
        pred = self.generate_aspects(review=review)
        return dspy.Prediction(aspects_with_label=pred.aspects_with_label)

In [51]:
uncompiled = DSPyInstructABSA()

In [52]:
pred = uncompiled(test_dspy[0].review)

ValueError: dictionary update sequence element #0 has length 1; 2 is required

# DSPy with Backend

In [18]:
model = "kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants"


In [19]:
dspy.JSONBackend

dspy.modeling.backends.json.JSONBackend